# COVID-19 Reopening / Recovery Indicators 

The NY Times wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html) and a further analysis showing how US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html). Vox has written about [the importance of COVID testing indicators](https://www.vox.com/2020/5/8/21249880/coronavirus-testing-covid-reopening-economy-lockdowns-social-distancing) as a prerequisite for reopening. 

The indicators used by New York State and Chicago are [provided in our GitHub](https://github.com/CityOfLosAngeles/aqueduct/blob/master/dags/public-health/covid19/Reopening_Indicators_Comparison.xlsx). 

This notebook gives helpful hints for working with our COVID-19 data in Jupyter Notebooks. It covers:
* How to import CSVs hosted on ESRI
* How to use functions `utils.py` to get grab the charts for any US county, state, or MSA.

### [White House Gating Criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
#### Symptoms
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

#### Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

#### Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

In [1]:
import pandas as pd
import utils

## Import data from ESRI CSV
The [README](https://github.com/CityOfLosAngeles/aqueduct/blob/master/dags/public-health/covid19/README.md#helpful-hints) gave some helpful hints for how to read in CSVs that are hosted by ESRI.

In [2]:
US_COUNTY_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/8aba663239fc428f8bcdc48e213e3172/data"
CROSSWALK_URL = (
    "https://raw.githubusercontent.com/CityOfLosAngeles/aqueduct/master/dags/public-health/covid19/"
    "msa_county_pop_crosswalk.csv"
)

In [3]:
df = pd.read_csv(US_COUNTY_URL, dtype={"fips": "str"})
df.head(2)

,Unnamed: 0,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths
0,62,Autauga,Alabama,01001,2020-03-24 07:00:00+00:00,32.539527,-86.644082,1,0.0,1.789901,NaN,242,0,1,0.0,46,0
1,63,Autauga,Alabama,01001,2020-03-25 07:00:00+00:00,32.539527,-86.644082,4,0.0,7.159606,NaN,381,1,3,0.0,139,1


In [4]:
crosswalk = pd.read_csv(CROSSWALK_URL)
crosswalk.head(2)

,cbsacode,cbsatitle,metro_micro,county,state,county_fips,fips_state_code,fips_county_code,county_pop,msa_pop
0,10100,"Aberdeen, SD",Micropolitan Statistical Area,Brown County,South Dakota,46013,46,13,38839.0,42668.0
1,10100,"Aberdeen, SD",Micropolitan Statistical Area,Edmunds County,South Dakota,46045,46,45,3829.0,42668.0


## County indicators for cases and deaths
* Uses the `case_indicators_county` function within `utils`
* 7-day rolling average shown for new cases
* 3-day rolling average shown for new deaths 

The function takes `COUNTY INFORMATION` and `START_DATE` as strings, and returns the charts and the dataframe used to create the chart.

`COUNTY_INFORMATION` takes form of {COUNTY_NAME, STATE_NAME} or {5-digit county FIPS code}:
* {COUNTY_NAME}: with our without " County" e.g. "Alameda County" or "Alameda"
* {STATE_NAME}: full state name or state abbreviation e.g. "California" or "CA"
* {5-DIGIT COUNTY FIPS CODE}: a [list of county FIPS found here.](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697) e.g. "06075" for San Francisco

`START_DATE` takes form of M/D/YY
* M: 1-12
* D: 1-31
* YY: last 2 digits of year

In [5]:
cook = utils.case_indicators_county("Cook, Illinois", "3/1/20")
cook.head()

alt.HConcatChart(...)

,county,state,state_abbrev,fips,date,Lat,Lon,cases,deaths,new_cases,new_deaths,cases_avg7,deaths_avg3,deaths_avg7
0,Cook,Illinois,IL,17031,2020-03-01 08:00:00+00:00,41.841448,-87.816588,3,0.0,1,0.0,NaN,NaN,NaN
1,Cook,Illinois,IL,17031,2020-03-02 08:00:00+00:00,41.841448,-87.816588,4,0.0,1,0.0,NaN,NaN,NaN
2,Cook,Illinois,IL,17031,2020-03-03 08:00:00+00:00,41.841448,-87.816588,4,0.0,0,0.0,NaN,0.0,NaN
3,Cook,Illinois,IL,17031,2020-03-04 08:00:00+00:00,41.841448,-87.816588,4,0.0,0,0.0,NaN,0.0,NaN
4,Cook,Illinois,IL,17031,2020-03-05 08:00:00+00:00,41.841448,-87.816588,5,0.0,1,0.0,NaN,0.0,NaN


Other acceptable examples are:
```
houston = utils.case_indicators_county("Houston, TX", "3/15/20")
atlanta = utils.case_indicators_county("Fulton, Georgia", "3/15/20")
```

## State indicators for cases and deaths
* Uses the `case_indicators_state` function within `utils`
* 7-day rolling average shown for new cases
* 3-day rolling average shown for new deaths 

The function takes `STATE_INFORMATION` and `START_DATE` as strings, and returns the charts and the dataframe used to create the chart.

`STATE_INFORMATION` takes form of {STATE_NAME}:
* {STATE_NAME}: full state name or state abbreviation e.g. "California" or "CA"

In [6]:
ny = utils.case_indicators_state("New York", "4/1/20")
ny.head()

alt.HConcatChart(...)

,state,state_abbrev,date,cases,deaths,new_cases,new_deaths,cases_avg7,deaths_avg3,deaths_avg7
0,New York,NY,2020-04-01 07:00:00+00:00,83948,3917,8115,710,NaN,NaN,NaN
1,New York,NY,2020-04-02 07:00:00+00:00,92506,4730,8558,813,NaN,NaN,NaN
2,New York,NY,2020-04-03 07:00:00+00:00,102987,5418,10481,688,NaN,737.000000,NaN
3,New York,NY,2020-04-04 07:00:00+00:00,113833,5991,10846,573,NaN,691.333333,NaN
4,New York,NY,2020-04-05 07:00:00+00:00,123160,6785,9327,794,NaN,685.000000,NaN


Other acceptable examples are:
```
tn = utils.case_indicators_state("Tennessee", "2/15/20")
```

## MSA indicators for cases and deaths
* Uses the `case_indicators_msa` function within `utils`
* 7-day rolling average shown for new cases
* 3-day rolling average shown for new deaths 

The function takes `METROPOLITAN_INFORMATION` and `START_DATE` as strings, and returns the charts and the dataframe used to create the chart. [Our crosswalk makes this information available.](https://github.com/CityOfLosAngeles/aqueduct/blob/master/dags/public-health/covid19/msa_county_pop_crosswalk.csv) 

`METROPOLITAN_INFORMATION` takes form of {CBSA NAME} or {CBSA CODE}:
* {CBSA NAME}: full metropolitan area name or any subset of it e.g. "Boston-Cambridge-Newton, MA-NH" or "Boston-Cambridge"
* {CBSA CODE}: e.g. "14460" for Boston-Cambridge-Newton, MA-NH or "31080" for Los Angeles-Long Beach-Anaheim, CA

In [7]:
boston_msa = utils.case_indicators_msa("Boston-Cambridge", "3/1/20")
boston_msa.head()

alt.HConcatChart(...)

,msa,msa_pop,date,cases,deaths,new_cases,new_deaths,cases_avg7,deaths_avg3,deaths_avg7
0,"Boston-Cambridge-Newton, MA-NH",4873019.0,2020-03-01 08:00:00+00:00,1,0.0,NaN,NaN,NaN,NaN,NaN
1,"Boston-Cambridge-Newton, MA-NH",4873019.0,2020-03-02 08:00:00+00:00,1,0.0,0.0,0.0,NaN,NaN,NaN
2,"Boston-Cambridge-Newton, MA-NH",4873019.0,2020-03-03 08:00:00+00:00,2,0.0,1.0,0.0,NaN,NaN,NaN
3,"Boston-Cambridge-Newton, MA-NH",4873019.0,2020-03-04 08:00:00+00:00,2,0.0,0.0,0.0,NaN,0.0,NaN
4,"Boston-Cambridge-Newton, MA-NH",4873019.0,2020-03-05 08:00:00+00:00,2,0.0,0.0,0.0,NaN,0.0,NaN


Other acceptable examples are:
```
dfw = utils.case_indicators_msa("Dallas-Fort Worth-Arlington, TX", "3/1/20")
durham = utils.case_indicators_msa("Durham-Chapel Hill", "4/1/20")
```

### References:
<br>US_COUNTY_URL = " http://lahub.maps.arcgis.com/sharing/rest/content/items/8aba663239fc428f8bcdc48e213e3172/data "
<br>LA_CITY_URL = " https://lahub.maps.arcgis.com/sharing/rest/content/items/7175fba373f541a7a19df56b6a0617f4/data "
<br>TESTING_URL = " http://lahub.maps.arcgis.com/sharing/rest/content/items/158dab4a07b04ecb8d47fea1746303ac/data "
<br>HOSPITAL_URL = " http://lahub.maps.arcgis.com/sharing/rest/content/items/3da1eb3e13a14743973c96b945bd1117/data "
<br>CROSSWALK_URL = " https://raw.githubusercontent.com/CityOfLosAngeles/aqueduct/master/dags/public-health/covid19/msa_county_pop_crosswalk.csv "

To convert to HTML, use this command in the terminal: 
`jupyter nbconvert --to html --no-input --no-prompt my-notebook.ipynb`